In [1]:
import sys
from collections import defaultdict

import unified_planning as up
import up_symk
from unified_planning.shortcuts import *
from unified_planning.io import PDDLReader

In [2]:
# Read a simple PDDL gripper problem from file

reader = PDDLReader()
pddl_problem = reader.parse_problem("gripper-domain.pddl", "gripper-prob01.pddl")
pddl_problem.add_quality_metric(MinimizeSequentialPlanLength())

# Validator
pv = PlanValidator(problem_kind=pddl_problem.kind)

In [3]:
# Solve problem optimally with SymK (bidirectional symbolic search)

with OneshotPlanner(name='symk') as planner:
    result = planner.solve(pddl_problem) # output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        pv_res = pv.validate(pddl_problem, result.plan)
        cost = pv_res.metric_evaluations[pddl_problem.quality_metrics[0]]
        print(f"{planner.name} found this plan: {result.plan} with cost {cost}.")
    else:
        print("No plan found.")

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 3 of `/tmp/ipykernel_79480/4014978636.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

SymK found this plan: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball1, roomb, left)
    drop(ball2, roomb, right)
    move(roomb, rooma)
    pick(ball3, rooma, left)
    pick(ball4, rooma, right)
    move(rooma, roomb)
    drop(ball3, roomb, left)
    drop(ball4, roomb, right) with cost 11.


In [4]:
# Find 5 optimal plans with SymK

plans_by_cost = defaultdict(lambda: [])

with AnytimePlanner(name='symk-opt', params={"number_of_plans": 5}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            pv_res = pv.validate(pddl_problem, result.plan)
            cost = pv_res.metric_evaluations[pddl_problem.quality_metrics[0]]
            plans_by_cost[cost].append(result.plan)
            print(f"Plan {i+1}: {result.plan} with cost {cost}.")
            print()
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            assert len(plans_by_cost) == 1
            for cost, plans in plans_by_cost.items():
                print(f"{planner.name} found {len(plans)} optimal plans with cost {cost}.")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.") 

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 5 of `/tmp/ipykernel_79480/3044654666.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

Plan 1: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball2, roomb, right)
    drop(ball1, roomb, left)
    move(roomb, rooma)
    pick(ball3, rooma, left)
    pick(ball4, rooma, right)
    move(rooma, roomb)
    drop(ball3, roomb, left)
    drop(ball4, roomb, right) with cost 11.

Plan 2: SequentialPlan:
    pick(ball2, rooma, right)
    pick(ball1, rooma, left)
    move(rooma, roomb)
    drop(ball2, roomb, right)
    drop(ball1, roomb, left)
    move(roomb, rooma)
    pick(ball3, rooma, left)
    pick(ball4, rooma, right)
    move(rooma, roomb)
    drop(ball4, r

In [5]:
# Query an anytime planner with OPTIMAL_PLANS guarantee and generate all optimal plans => SymK

plans_by_cost = defaultdict(lambda: [])

with AnytimePlanner(problem_kind=pddl_problem.kind, anytime_guarantee="OPTIMAL_PLANS") as planner:
    print(planner.name)
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            pv_res = pv.validate(pddl_problem, result.plan)
            cost = pv_res.metric_evaluations[pddl_problem.quality_metrics[0]]
            plans_by_cost[cost].append(result.plan)
            if i > 0 and i % 100 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            assert len(plans_by_cost) == 1
            for cost, plans in plans_by_cost.items():
                print(f"{planner.name} found {len(plans)} optimal plans with cost {cost}.")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.") 

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 5 of `/tmp/ipykernel_79480/3535417708.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

SymK (with optimality guarantee)
SymK (with optimality guarantee) found 100 plans...
SymK (with optimality guarantee) found 200 plans...
SymK (with optimality guarantee) found 300 plans...
SymK (with optimality guarantee) found 384 optimal plans with cost 11.


In [6]:
# Find 500 plans with SymK

plans_by_cost = defaultdict(lambda: [])

with AnytimePlanner(name='symk', params={"number_of_plans": 500}) as planner:
    for i, result in enumerate(planner.get_solutions(pddl_problem)): # output_stream=sys.stdout): 
        if result.status == up.engines.PlanGenerationResultStatus.INTERMEDIATE:
            pv_res = pv.validate(pddl_problem, result.plan)
            cost = pv_res.metric_evaluations[pddl_problem.quality_metrics[0]]
            plans_by_cost[cost].append(result.plan)
            if i > 0 and i % 100 == 0:
                print(f"{planner.name} found {i} plans...")
        elif result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print()
            print(f"{planner.name} found {i} plans!")
            for cost, plans in plans_by_cost.items():
                print(f"{planner.name} found {len(plans)} plans with cost {cost}.")
        elif result.status not in unified_planning.engines.results.POSITIVE_OUTCOMES:
            print("No plan found.") 

  *** Credits ***
  * In operation mode `AnytimePlanner` at line 5 of `/tmp/ipykernel_79480/685317883.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent classical optimal and top-k planner.

SymK found 100 plans...
SymK found 200 plans...
SymK found 300 plans...
SymK found 400 plans...

SymK found 500 plans!
SymK found 384 plans with cost 11.
SymK found 116 plans with cost 12.
